In [ ]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from google.colab import drive

print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

In [ ]:
drive.mount('/content/gdrive')

RAW_DATA_PATH = '/content/gdrive/My Drive/Colab Notebooks/content/Human'
MODEL_SAVE_PATH = '/content/gdrive/My Drive/Colab Notebooks/content/Human/models'

TRAIN_PATH = os.path.join(RAW_DATA_PATH, 'train')
VALID_PATH = os.path.join(RAW_DATA_PATH, 'valid')
TEST_PATH = os.path.join(RAW_DATA_PATH, 'test')

os.makedirs(MODEL_SAVE_PATH, exist_ok=True)

print(f"Train path: {TRAIN_PATH}")
print(f"Valid path: {VALID_PATH}")
print(f"Test path: {TEST_PATH}")

In [ ]:
def reorganize_for_keras(base_path):
    person_folder = os.path.join(base_path, 'person')
    
    if os.path.exists(person_folder):
        print(f"Already organized: {base_path}")
        return
    
    os.makedirs(person_folder, exist_ok=True)
    
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')
    moved_count = 0
    
    for filename in os.listdir(base_path):
        if filename.lower().endswith(image_extensions):
            src = os.path.join(base_path, filename)
            dst = os.path.join(person_folder, filename)
            shutil.move(src, dst)
            moved_count += 1
    
    print(f"Moved {moved_count} images to {person_folder}")

reorganize_for_keras(TRAIN_PATH)
reorganize_for_keras(VALID_PATH)
reorganize_for_keras(TEST_PATH)

In [ ]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
EPOCHS = 50

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    VALID_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

NUM_CLASSES = len(train_generator.class_indices)
print(f"Classes: {train_generator.class_indices}")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Train samples: {train_generator.samples}")
print(f"Valid samples: {valid_generator.samples}")
print(f"Test samples: {test_generator.samples}")

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation='softmax' if NUM_CLASSES > 1 else 'sigmoid')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy' if NUM_CLASSES > 1 else 'binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    os.path.join(MODEL_SAVE_PATH, 'best_model.h5'),
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=valid_generator,
    callbacks=[early_stopping, checkpoint]
)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.tight_layout()
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
final_model_path = os.path.join(MODEL_SAVE_PATH, 'human_classifier.h5')
model.save(final_model_path)
print(f"Model saved to: {final_model_path}")

In [ ]:
from google.colab import files
from tensorflow.keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
    path = '/content/' + fn
    img = image.load_img(path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0

    prediction = model.predict(x)
    print(f"File: {fn}")
    print(f"Prediction: {prediction}")
    print(f"Predicted class: {np.argmax(prediction)}")
    print()